# Challenge TextRecognition Apprentissage Profond
Auteur : JMLD - AD - CN
Date : 30/01/2017

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import gensim
import pickle
import logging

import zipfile
import collections

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import random

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

In [2]:
datapath = "../IntroductionDL/"
trainf = "calldesk-intents.txt"
dataf = datapath + trainf
train = pd.read_csv(dataf,names=['Cible','Mot1','Mot2','Mot3','Mot4','Mot5','Mot6','Mot7','Mot8','Mot9','Mot10'],dtype=str,delim_whitespace=True)
train.dtypes

Cible    object
Mot1     object
Mot2     object
Mot3     object
Mot4     object
Mot5     object
Mot6     object
Mot7     object
Mot8     object
Mot9     object
Mot10    object
dtype: object

In [3]:
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)  # %(asctime)s

In [4]:
model = gensim.models.Word2Vec(size=200, sg=1, alpha=0.05, window=5, batch_words=1000, min_count=1, workers=2, negative=10, iter=10)

In [5]:
sentences = gensim.models.word2vec.LineSentence('calldesk-intents.txt')

In [6]:
model.build_vocab(sentences)

INFO : collecting all words and their counts
INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO : collected 342 word types from a corpus of 6687 raw words and 800 sentences
INFO : Loading a fresh vocabulary
INFO : min_count=1 retains 342 unique words (100% of original 342, drops 0)
INFO : min_count=1 leaves 6687 word corpus (100% of original 6687, drops 0)
INFO : deleting the raw counts dictionary of 342 items
INFO : sample=0.001 downsamples 56 most-common words
INFO : downsampling leaves estimated 2586 word corpus (38.7% of prior 6687)
INFO : estimated required memory for 342 words and 200 dimensions: 718200 bytes
INFO : resetting layer weights


In [7]:
model.train(sentences)

INFO : training model with 2 workers on 342 vocabulary and 200 features, using sg=1 hs=0 sample=0.001 negative=10 window=5
INFO : expecting 800 sentences, matching count from corpus used for vocabulary survey
INFO : worker thread finished; awaiting finish of 1 more threads
INFO : worker thread finished; awaiting finish of 0 more threads
INFO : training on 66870 raw words (25848 effective words) took 0.2s, 158492 effective words/s


25848

In [8]:
model.save('./' + 'calldesk-intents-model')

INFO : saving Word2Vec object under ./calldesk-intents-model, separately None
INFO : not storing attribute cum_table
INFO : not storing attribute syn0norm
INFO : saved ./calldesk-intents-model


In [9]:
# Extract vocabulary dictionnary:
vocabulary_dict = {}
for key, value in model.wv.vocab.items():
    vocabulary_dict[str(key)] = int(str(value).split("index:")[1].split(",", 1)[0])
vocabulary_dict

{'$NUMBER$': 216,
 '$TIME$': 0,
 '15h': 240,
 'Confirm': 29,
 'Infirm': 14,
 'NewAppointment': 1,
 'OK': 319,
 'Propose': 9,
 'S': 306,
 'TIME': 110,
 'Transfer': 55,
 'a': 62,
 'a-t-il': 229,
 'absolument': 59,
 'accessible': 202,
 'accord': 254,
 'ah': 67,
 'ai': 47,
 'aimerai': 219,
 'aimerais': 48,
 'aimé': 230,
 'ajouter': 260,
 'aller': 106,
 'allez': 107,
 'alors': 125,
 'amour': 313,
 'annonce': 308,
 'appelle': 22,
 'après': 149,
 'arrangerait': 338,
 'arrive': 303,
 'as': 187,
 'attendez': 127,
 'attends': 259,
 'au': 76,
 'aurai': 327,
 'aurais': 137,
 'aurez': 258,
 'auriez-vous': 283,
 'aussi': 333,
 'autant': 299,
 'autour': 245,
 'autre': 188,
 'autres': 322,
 'avais': 199,
 'avec': 12,
 'avez': 44,
 'avez-vous': 86,
 'avoir': 143,
 'bah': 167,
 'beaucoup': 104,
 'ben': 144,
 'besoin': 273,
 'bien': 51,
 'bon': 84,
 'bonjour': 18,
 'bonne': 200,
 'c': 24,
 'ca': 88,
 'cabinet': 122,
 'car': 252,
 'cas': 284,
 'ce': 79,
 'cherche': 255,
 'chez': 164,
 'choix': 323,
 'chos

In [10]:
# Extract embedding matrix:
embedding_matrix = model.wv.syn0
embedding_matrix.shape
##print(embedding_matrix[0, :5])

(342, 200)

In [11]:
#print(model.vocab)
model.wv.vocab.items()

dict_items([('jouons', <gensim.models.word2vec.Vocab object at 0x11effa668>), ('à', <gensim.models.word2vec.Vocab object at 0x11effa358>), ('avais', <gensim.models.word2vec.Vocab object at 0x11effa400>), ('durand', <gensim.models.word2vec.Vocab object at 0x11effa2b0>), ('appelle', <gensim.models.word2vec.Vocab object at 0x11effa128>), ('jamais', <gensim.models.word2vec.Vocab object at 0x11effa438>), ('bonne', <gensim.models.word2vec.Vocab object at 0x11effa080>), ('sais', <gensim.models.word2vec.Vocab object at 0x11effa470>), ('peux', <gensim.models.word2vec.Vocab object at 0x11effa4e0>), ('voir', <gensim.models.word2vec.Vocab object at 0x11effa4a8>), ('serait-ce', <gensim.models.word2vec.Vocab object at 0x11effa518>), ('accessible', <gensim.models.word2vec.Vocab object at 0x11effa550>), ('personnes', <gensim.models.word2vec.Vocab object at 0x11effa588>), ('classe', <gensim.models.word2vec.Vocab object at 0x11effa5c0>), ('préférerai', <gensim.models.word2vec.Vocab object at 0x11effa5f8

In [71]:
np.random.seed(1350)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, MaxPooling1D, Embedding
from keras.utils import np_utils

In [72]:
embedding_layer = Embedding(342, 200, weights=[embedding_matrix], trainable=False, input_shape=xtrain.shape) 

In [73]:
model2 = Sequential()
model2.add(embedding_layer)

In [74]:
model2.add(Convolution1D(nb_filter=10, filter_length=10, border_mode='valid', activation='relu', subsample_length=1))

In [75]:
model2.add(MaxPooling1D())

In [76]:
model2.add(Dense(10))
model2.add(Dropout(0.2))
model2.add(Activation('relu'))

In [77]:
model2.add(Dense(1))
model2.add(Activation('sigmoid'))

In [78]:
model2.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [79]:
import random
def kfolds(k,N,seed=None):
    random.seed(seed)
    out = [ list() for _ in range(k) ]
    for n in range(N): out[random.randrange(k)].append(n)
    return(out)

K = kfolds(seed=42,k=5,N=len(train))
fold1 = sum(K[1:5],[]) # K[1] + K[2] + K[3] + K[4]
#fold1 = K[1]+K[2]+K[3]+K[4]
xtrain = train.iloc[fold1][train.keys()[1:]]
ytrain = train.iloc[fold1][train.keys()[0]]

xtest = train.iloc[K[0]][train.keys()[1:]]
ytest = train.iloc[K[0]][train.keys()[0]]

In [81]:
model2.fit(xtrain, ytrain, batch_size=342, nb_epoch=2, validation_data=(xtest, ytest))

ValueError: Error when checking model target: expected activation_10 to have 3 dimensions, but got array with shape (642, 1)

## Label Encoder / Bag of Words + ML

In [24]:
train.head()

,Cible,Mot1,Mot2,Mot3,Mot4,Mot5,Mot6,Mot7,Mot8,Mot9,Mot10
0,Confirm,OK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Confirm,absolument,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Confirm,ah,oui,ça,c,est,bien,ça,comme,ça,NaN
3,Confirm,aller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Confirm,allez,c,est,bon,pour,$TIME$,NaN,NaN,NaN,NaN


In [25]:
dataf

'../IntroductionDL/calldesk-intents.txt'

In [5]:
le.fit(train['Cible'])
list(le.classes_)
train['Cible'] = le.transform(train['Cible'])

#pd.get_dummies
train_Mot1 = pd.get_dummies(train['Mot1'])
train_Mot2 = pd.get_dummies(train['Mot2'])
train_Mot3 = pd.get_dummies(train['Mot3'])
train_Mot4 = pd.get_dummies(train['Mot4'])
train_Mot5 = pd.get_dummies(train['Mot5'])
train_Mot6 = pd.get_dummies(train['Mot6'])
train_Mot7 = pd.get_dummies(train['Mot7'])
train_Mot8 = pd.get_dummies(train['Mot8'])
train_Mot9 = pd.get_dummies(train['Mot9'])
train_Mot10 = pd.get_dummies(train['Mot10'])
train_new = pd.concat([train,train_Mot1,train_Mot2,train_Mot3,train_Mot4,train_Mot5,train_Mot6,train_Mot7,train_Mot8,train_Mot9,train_Mot10],axis=1)

#Suppression des doublons

del train_new['Mot1']
del train_new['Mot2']
del train_new['Mot3']
del train_new['Mot4']
del train_new['Mot5']
del train_new['Mot6']
del train_new['Mot7']
del train_new['Mot8']
del train_new['Mot9']
del train_new['Mot10']


#del train_new['Finess']

In [6]:
train_new.head()

,Cible,$TIME$,OK,a-t-il,absolument,accessible,ah,ajouter,aller,allez,...,prochain,prochains,que,rendez-vous,revoir,s,te,une,voir,vous
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train_new.iloc[2][train_new.iloc[2]>0]

ah       1
oui      1
ça       1
c        1
est      1
bien     1
ça       1
comme    1
ça       1
Name: 2, dtype: int64

In [8]:
import random
def kfolds(k,N,seed=None):
    random.seed(seed)
    out = [ list() for _ in range(k) ]
    for n in range(N): out[random.randrange(k)].append(n)
    return(out)

K = kfolds(seed=42,k=5,N=len(train_new))
fold1 = sum(K[1:5],[]) # K[1] + K[2] + K[3] + K[4]
#fold1 = K[1]+K[2]+K[3]+K[4]
xtrain = train_new.iloc[fold1][train_new.keys()[1:]]
ytrain = train_new.iloc[fold1][train_new.keys()[0]]

xtest = train_new.iloc[K[0]][train_new.keys()[1:]]
ytest = train_new.iloc[K[0]][train_new.keys()[0]]

In [9]:
from sklearn.ensemble import RandomForestClassifier
#
clf = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)

In [10]:
fitclf = clf.fit(xtrain,ytrain)

In [11]:
fitclf.score(xtest,ytest)

0.87341772151898733

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()

In [13]:
fitgbc = gbc.fit(xtrain,ytrain)

In [14]:
fitgbc.score(xtest,ytest)

0.83544303797468356

# Use of nltk to improve the data cleaning

In [28]:
import nltk
from nltk import word_tokenize
#nltk.download()

In [29]:
import snowballstemmer

stemmer = snowballstemmer.stemmer('french');
print(stemmer.stemWords("Nous sommes le monde".split()));

['Nous', 'somm', 'le', 'mond']


# Word2vec method

Training

In [15]:
import word2vec

In [19]:
#Create better input for word2vec
word2vec.word2phrase(dataf, datapath + "calldesk-intents-phrases.txt" , verbose=True)

In [20]:
word2vec.word2clusters(datapath + "calldesk-intents-phrases.txt", datapath + "calldesk-intents-clusters.txt", 100, verbose=True)

b'Starting training using file ../IntroductionDL/calldesk-intents-phrases.txt\n'b'Vocab size: 108\n'b'Words in train file: 6991\n'

Predictions

In [31]:
model = word2vec.load_word2vec_format(datapath + "calldesk-intents-clusters.txt", binary=False)

AttributeError: module 'word2vec' has no attribute 'load_word2vec_format'

# TensorFlow Word2Vec

In [39]:
import math

In [40]:
vocabulary_size = 50000
embedding_size = 200

In [41]:
embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
nce_weights = tf.Variable(
  tf.truncated_normal([vocabulary_size, embedding_size],
                      stddev=1.0 / math.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

In [37]:
embeddings